In [ ]:
!pip install optunity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 7.5 MB/s 
  Created wheel for optunity: filename=Optunity-1.1.1-py3-none-any.whl size=72031 sha256=d2417ece96899f69a58b5570e1bc0c6870b696a0b084e41c21dc8ebbd471f700
  Stored in directory: /root/.cache/pip/wheels/d1/03/a8/c725421da03e2bb8b33fb20b7159727fb1d47806f0ae844a91
Successfully built optunity


In [ ]:
!pip install Sobol

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for Sobol: filename=sobol-0.9-py3-none-any.whl size=7923 sha256=c4c134989834336b1d7a0e4fafe7bd1e619fff77b42410928257d01d1a2774b7
  Stored in directory: /root/.cache/pip/wheels/a8/00/1c/7934462518942429eaaed55b9268963bb5c8b7553147cf72b2
Successfully built Sobol


In [ ]:
import math
import operator as op
import random
import numpy as np
import array
import functools
import itertools
from optunity import functions as fun
from optunity import solvers
from optunity.solvers import solver_registry
from optunity import util
from optunity.solvers.solver_registry import register_solver
from optunity.solvers.util import Solver, _copydoc
from optunity.solvers.util import uniform_in_bounds, scale_unit_to_bounds
from optunity.solvers.Sobol import Sobol
@register_solver('salp swarm',
                 'salp swarm optimization',
                 ['Maximizes the function using salp swarm optimization.',
                  ' ',
                  'This is a two-phase approach:',
                  '1. Initialization: randomly initializes num_salps salps.',
                  '   Salps are randomized uniformly within the box constraints.',
                  '2. Iteration: salps move during num_generations iterations.',
                  '   Movement is based on their mutual attractions .',
                  ' ',
                  'This function requires the following arguments:',
                  '- num_salps: number of salps to use in the swarm',
                  '- num_generations: number of iterations used by the swarm',
                  '- box constraints via key words: constraints are lists [lb, ub]', ' ',
                  'This solver performs num_salps*num_generations function evaluations.'
                  ])
class SalpSwarm(Solver):
    """
    .. include:: /global.rst
    Refer to |sso| for details on this algorithm.
    """
    class Salp:
        def __init__(self, SalpPositions, FoodPosition,  best, SalpFitness, best_fitness):
            """Constructs a Salp."""
            self.SalpPositions = SalpPositions
            self.FoodPosition = FoodPosition

            self.best = best
            self.SalpFitness = SalpFitness
            self.best_fitness = best_fitness
        def clone(self):
            """Clones this Salp."""
            return SalpSwarm.Salp(SalpPositions=self.SalpPositions[:],FoodPosition=self.FoodPosition[:], best=self.best[:], SalpFitness=self.SalpFitness,
                                    best_fitness=self.best_fitness)
        def __str__(self):
            string = 'Salp{SalpPositions=' + str(self.SalpPositions)
            string += ', FoodPosition=' + str(self.FoodPosition)
            string += ', best=' + str(self.best)
            string += ', SalpFitness=' + str(self.SalpFitness)
            string += ', best_fitness=' + str(self.best_fitness)
            string += '}'
            return string
    def __init__(self, num_salps=10, num_generations=2, lb=None, ub=None, dim=8, c2=1.5, c3=1.0, **kwargs):
        """
        Initializes a SSO solver.
        :param num_salps: number of salps to use
        :type num_salps: int
        :param num_generations: number of generations to use
        :type num_generations: int
        :param lb: parameter used in updating position of leader
        :type lb: int
        :param ub: parameter used in updating position of leader
        :type ub: int
        :param dim: parameter used in updating position of leader
        :type dim: int
        :param c2: parameter used in updating position of leader
        :type c2: float
        :param c3: parameter used in updating position of leader
        :type c3: float
        :param kwargs: box constraints for each hyperparameter
        :type kwargs: {'name': [lb, ub], ...}
        The number of function evaluations it will perform is `num_generations`.
        The search space is rescaled to the unit hypercube before the solving process begins.
        >>> solver = SalpSwarm(num_salps=40, num_generations=10, x=[-1, 16], y=[0, 64])
        >>> solver.bounds['x']
        [-1, 16]
        >>> solver.bounds['y']
        [0, 64]
        >>> solver.num_salps
        40
        >>> solver.num_generations
        10
        .. warning:: |warning-unconstrained|
        """
        assert all([len(v) == 2 and v[0] <= v[1]
                    for v in kwargs.values()]), 'kwargs.values() are not [lb, ub] pairs'
        self._bounds = kwargs
        self._num_salps = num_salps
        self._num_generations = num_generations
        self._dim = dim
        if dim is None:
            self._dim = len(kwargs) * 4
        self._ub = [(b[1]) for _, b in self.bounds.items()]
        self._lb = [(b[0]) for _, b in self.bounds.items()]
        self._sobolseed = random.randint(100,2000)
        self._c2 = c2
        self._c3 = c3
    @property
    def c2(self):
        return self._c2

    @property
    def c3(self):
        return self._c3
    @property
    def sobolseed(self):
       return self._sobolseed

    @sobolseed.setter
    def sobolseed(self, value): self._sobolseed = value
    @staticmethod
    def suggest_from_box(num_evals, **kwargs):
        """Create a configuration for a SalpSwarm solver.
        :param num_evals: number of permitted function evaluations
        :type num_evals: int
        :param kwargs: box constraints
        :type kwargs: {'param': [lb, ub], ...}
        >>> config = SalpSwarm.suggest_from_box(200, x=[-1, 16], y=[0, 64])
        >>> config['x']
        [-1, 16]
        >>> config['y']
        [0, 64]
        >>> config['num_salps'] > 0
        True
        >>> config['num_generations'] > 0
        True
        >>> solver = SalpSwarm(**config)
        >>> solver.bounds['x']
        [-1, 16]
        >>> solver.bounds['y']
        [0, 64]
        """
        d = dict(kwargs)
        if num_evals > 1000:
            d['num_salps'] = 100
        elif num_evals >= 200:
            d['num_salps'] = 20
        elif num_evals >= 10:
            d['num_salps'] = 10
        else:
            d['num_salps'] = num_evals
        d['num_generations'] = int(math.ceil(float(num_evals) / d['num_salps']))
        return d
    @property
    def num_salps(self):
        return self._num_salps
    @property
    def num_generations(self):
        return self._num_generations
    @property
    def dim(self):
        return self._dim
    @property
    def ub(self):
        return self._ub
    @property
    def lb(self):
        return self._lb
    @property
    def bounds(self):
        return self._bounds
    def generate(self):
        """Generate a new Salp."""
        if len(self.bounds) < Sobol.maxdim():
            sobol_vector, self.sobolseed = Sobol.i4_sobol(len(self.bounds), self.sobolseed)
            vector = optunity.solvers.util.scale_unit_to_bounds(sobol_vector, self.bounds.values()) 
        else: vector = optunity.solvers.util.uniform_in_bounds(self.bounds)
        
        sal =SalpSwarm.Salp(SalpPositions=array.array('d', vector), FoodPosition=array.array('d', vector),
                             SalpFitness=None, best=None, best_fitness=None)
        return sal

    def updateSalp(self, sal, best, c2, c3):
        sal.SalpPositions = np.zeros((self.num_salps, self.dim))
        for i in range(self.dim):
          sal.SalpPositions[:, i] = np.random.uniform(0, 1, self.num_salps) * (self.ub[0] - self.lb[0]) + self.lb[0]
        sal.SalpFitness = np.full(self.num_salps, float("inf"))
        sal.FoodPosition = np.zeros(self.dim)
        FoodFitness = float("inf")
        sorted_salps_fitness = np.sort(sal.SalpFitness)
        I = np.argsort(sal.SalpFitness)
        Sorted_salps = np.copy(sal.SalpPositions[I, :])
        sal.FoodPosition = np.copy(Sorted_salps[0, :])
        FoodFitness = sorted_salps_fitness[0]
        """Update the salp."""
        Iteration = 1
        Max_iteration=3
        c1 = 2 * math.exp(-((4 * Iteration / Max_iteration) ** 2))
        for i in range(0, self.num_salps):
            sal.SalpPositions = np.transpose(sal.SalpPositions)
            if i < self.num_salps / 2:
                for j in range(0, self.dim):
                    c2 = random.random()
                    c3 = random.random()
                    # Eq. (3.1) in the paper
                    if c3 < 0.5:
                        sal.SalpPositions[j, i] = sal.FoodPosition[j] + c1 * (
                            (self.ub[0] - self.lb[0]) * c2 + self.lb[0])
                    else:
                        sal.SalpPositions[j, i] = sal.FoodPosition[j] - c1 * (
                            (self.ub[0] - self.lb[0]) * c2 + self.lb[0])

                    ####################

            elif i >= self.num_salps/ 2 and i < self.num_salps + 1:
                point1 = sal.SalpPositions[:, i - 1]
                point2 = sal.SalpPositions[:, i]
                sal.SalpPositions[:, i] = (point2 + point1) / 2
            sal.SalpPositions = np.transpose(sal.SalpPositions)

        for i in range(0, self.num_salps):
          # Check if salps go out of the search spaceand bring it back
          for j in range(self.dim):
            sal.SalpPositions[i, j] = np.clip(sal.SalpPositions[i, j], self.lb[0], self.ub[0])
            if sal.SalpFitness[i] < FoodFitness:
              sal.FoodPosition = np.copy(sal.SalpPositions[i, :])
              FoodFitness = sal.SalpFitness[i]
        
    def salp2dict(self, salp):
        return dict([(k, v) for k, v in zip(self.bounds.keys(), salp.SalpPositions)])
    @_copydoc(Solver.optimize)

    def optimize(self, f, maximize=True, pmap=map):
        @functools.wraps(f)
        def evaluate(d):
            return f(**d)
        if maximize:
            fit = 1.0
        else:
            fit = -1.0
        pop = [self.generate() for _ in range(self.num_salps)]
        best = None
        for g in range(self.num_generations):
          fitnesses= pmap(evaluate, list(map(self.salp2dict, pop)))
        for sal, SalpFitness in zip(pop, fitnesses):
          sal.SalpFitness = fit * optunity.solvers.util.score(SalpFitness)
          if not sal.best or sal.best_fitness < sal.SalpFitness:
            sal.best = sal.SalpPositions
            sal.best_fitness = sal.SalpFitness
          if not best or best.SalpFitness < sal.SalpFitness:
             best= sal.clone()
        for sal in pop:
             self.updateSalp(sal, best, self.c2, self.c3)
        return dict([(k, v)
                        for k, v in zip(self.bounds.keys(), best.SalpPositions)]), None

